In [12]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC/New_traces/traces/aligned/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_New_traces.json"
num_ues_list = [5, 10, 15, 20, 25]
embb_prb_list = [1, 3, 9, 27, 81]
urllc_prb_list = [1, 3, 9, 27, 81]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 2750 training configurations and saved to xapp_dqn_training_configs_New_traces.json


## 20 seconds

In [13]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC/Trace_20s/traces/aligned/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s.json"
num_ues_list = [5, 10, 15, 20, 25]
embb_prb_list = [1, 3, 9, 27, 81]
urllc_prb_list = [1, 3, 9, 27, 81]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 2750 training configurations and saved to xapp_dqn_training_configs_Trace_20s.json


### Simpler training

In [14]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC/Trace_20s/traces/aligned/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_simpler.json"
num_ues_list = [1,3,5,7,9]
embb_prb_list = [1, 5, 7, 9, 11]
urllc_prb_list = [1, 5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 2750 training configurations and saved to xapp_dqn_training_configs_Trace_20s_simpler.json


## Config for test

In [ ]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC/New_traces/traces/aligned/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_New_traces.json"
num_ues_list = [2, 4, 8, 16, 20]
embb_prb_list = [3]
urllc_prb_list = [3]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")
